## Utilisation des classes ( programmation orienté-objet)




In [8]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=7f8272ff57813de2250e979a2b4f7419bb2aa2b7b18ac871cb055b1c2499db52
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [9]:
import datetime as dt 
import requests
import json
import pandas as pd 
from pandasql import sqldf

In [10]:
API_KEY = "e5b7d494b86c8fdab7619f416c77b12a"  #météo
CITY = "Toulouse"
URL_Energy = 'https://tinyurl.com/4f95debc'

In [11]:
#convertir une température  Kelvin en Celsius
def kelvin_to_celsius_fahrenheit(kelvin):
    
    celsuis = kelvin - 273.15 
    return celsuis

In [12]:
#Adaptateur pour l'API météo
class WeatherAdapter:
    def __init__(self, api_key):
        self.base_url = "http://api.openweathermap.org/data/2.5/weather?"
        self.api_key = api_key
    
    def get_weather(self, city):
        url = self.base_url + "q=" + city + "&appid=" + self.api_key
        response = requests.get(url)
        return response.json()
    
#Adaptateur pour l'API Consommation d'énergie
class EnergyAdapter:
    def __init__(self, base_url):
        self.base_url = base_url
    
    def get_energy_production(self):
        req = requests.get(self.base_url)

        data = json.loads(req.content)

   
        #response = requests.get(self.base_url + "/" + region)
        return data

# Classe pour le médiateur 
class InformationMediator:
    def __init__(self, weather_adapter, energy_adapter):
        self.weather_adapter = weather_adapter
        self.energy_adapter = energy_adapter
    
    def get_combined_info(self, city):
        weather_data = self.weather_adapter.get_weather(city)
        data = self.energy_adapter.get_energy_production()
        dict_info = {
            
            "id" : weather_data['id'],
            "temp_celsius" : round(kelvin_to_celsius_fahrenheit(weather_data["main"]["temp"]),2),
            "wind_speed": weather_data['wind']['speed'],
            "temperature": weather_data["main"]["temp"],
            "humidity" : weather_data['main']["humidity"],
            "lon" : weather_data['coord']["lon"],
            "lat" : weather_data['coord']["lat"],
            "country" : weather_data['sys']["country"],
            "cod" : weather_data['cod'],
            "sunrise_time": dt.datetime.utcfromtimestamp(weather_data['sys']['sunrise'] + weather_data['timezone']),
            "sunset_time" : dt.datetime.utcfromtimestamp(weather_data['sys']['sunset'] + weather_data['timezone']),
            "description": weather_data["weather"][0]["description"],
            "region" : data["records"][7]["fields"]["region"],
            "code_insee_region" : data["records"][7]["fields"]["code_insee_region"],
            "consommation_brute_totale" :  data["records"][7]["fields"]["consommation_brute_totale"],
            "consommation_brute_electricite_rte" : data["records"][7]["fields"]["consommation_brute_electricite_rte"],
            "date" : data["records"][7]["fields"]["date"],
            "heure": data["records"][7]["fields"]["heure"]
            
        }
        
        
    
        df = pd.DataFrame(dict_info,index=[0])
        #return df.transpose()
        return df

In [13]:
weatherAdapter  = WeatherAdapter(API_KEY)
energyAdapter = EnergyAdapter(URL_Energy)

informationMediator = InformationMediator(weatherAdapter,energyAdapter)


Exemple de queries:

'' Savoir la consommation d'énergie brute totale et la consommation brute d'électricité ''

1) query = "SELECT country, region, consommation_brute_totale, consommation_brute_electricite_rte FROM df"

'' Savoir la température et l'humidité sur Occitanie (Toulouse)''

2) query = "SELECT temp_celsius ,temperature, humidity, wind_speed FROM df"

'' Savoir la température et la consommation d'énergie brute totale sur Occitanie (Toulouse)''

3) query = "SELECT temp_celsius ,consommation_brute_totale FROM df"

In [17]:
query = input()
def requetes(query):
  informationMediator = InformationMediator(weatherAdapter,energyAdapter)
  df = informationMediator.get_combined_info(CITY)
  selected_columns_df = sqldf(query)
  print("Sur Occitanie (Toulouse), on a ces informations : ")
  return selected_columns_df.transpose()
selected_columns_df = requetes(query)
print(selected_columns_df)

SELECT temp_celsius ,consommation_brute_totale FROM df
Sur Occitanie (Toulouse), on a ces informations : 
                                 0
temp_celsius                  9.02
consommation_brute_totale  5070.00
